In [55]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [56]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

x = dados[['preco', 'idade_do_modelo','km_por_ano']]
y = dados[['vendido']]
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size=0.25, stratify=y)
print("treinamento com %d elementos e testaremos com %d elementos" %(len(treino_x), len(teste_x)))

treinamento com 7500 elementos e testaremos com 2500 elementos


In [57]:
from sklearn.dummy import DummyClassifier
SEED = 158020
np.random.seed(SEED)
dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia =dummy_stratified.score(teste_x, teste_y) * 100
print("A Acurácia do dummy stratified for %.2f%%" % acuracia)

A Acurácia do dummy stratified for 58.00%


In [58]:
from sklearn.tree import DecisionTreeClassifier
SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=1)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)
acuracia = accuracy_score(teste_y, previsoes) * 100
print("A Acurácia foi %.2f%%" % acuracia)

A Acurácia foi 77.28%


In [59]:
def imprime_resulados(result):
  media = result['test_score'].mean()
  desvio_padrao = result['test_score'].std()
  print('Accuracy média:  %.2f' %(media * 100))
  print("Accuracy intervalo:  [%.2f %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) *100))

In [60]:
from sklearn.model_selection import cross_validate
SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
result = cross_validate(modelo, x, y, cv=5, return_train_score=False)
imprime_resulados(result)


Accuracy média:  75.78
Accuracy intervalo:  [75.21 76.35]


In [78]:
dados_azar = dados.sort_values(by=['vendido'], ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
4999,74023.29,0,12,24812.80412,15
5322,84843.49,0,13,23095.63834,13
5319,83100.27,0,19,36240.72746,23
5316,87932.13,0,16,32249.56426,16
5315,77937.01,0,15,28414.50704,17


In [72]:
from sklearn.model_selection import KFold
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
result = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False)
imprime_resulados(result)


Accuracy média:  57.84
Accuracy intervalo:  [34.29 81.39]


In [73]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
result = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False)
imprime_resulados(result)

Accuracy média:  75.78
Accuracy intervalo:  [72.30 79.26]


In [74]:
from sklearn.model_selection import StratifiedKFold
SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
result = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False)
imprime_resulados(result)

Accuracy média:  75.78
Accuracy intervalo:  [73.55 78.01]


#### Gerando dados aleatório de modelo de carros para simulação de agrupamento

In [75]:
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1

dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [76]:
dados.modelo.value_counts()

20    901
19    798
18    771
21    723
17    709
16    668
14    621
22    575
15    573
13    557
12    511
11    401
10    371
23    370
9     336
8     278
7     206
24    199
6     181
5     108
4      76
3      44
2      17
1       6
Name: modelo, dtype: int64

#### Testando a validação cruzada com GroupKFold

In [77]:
from sklearn.model_selection import GroupKFold
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
result = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False, groups=dados.modelo)
imprime_resulados(result)

Accuracy média:  75.78
Accuracy intervalo:  [73.67 77.90]


#### Cross Validation with StandardScaler

In [84]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A Acurácia foi %.2f%%" % acuracia)

c:\Users\rober\Desktop\machine_learning\.venv\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


A Acurácia foi 77.12%


In [85]:
from sklearn.model_selection import GroupKFold
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = SVC()
result = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False, groups=dados.modelo)
imprime_resulados(result)

Accuracy média:  77.27
Accuracy intervalo:  [74.35 80.20]


In [86]:
from sklearn.pipeline import Pipeline
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])
cv = GroupKFold(n_splits=10)
result = cross_validate(pipeline, x_azar, y_azar, cv=cv, return_train_score=False, groups=dados.modelo)
imprime_resulados(result)

Accuracy média:  76.68
Accuracy intervalo:  [74.28 79.08]
